In [1]:
from selenium import webdriver
import bs4 as bs
from bs4 import BeautifulSoup
import sys
import time
import os
import csv
import re
import pandas as pd
import random

In [25]:
df = pd.DataFrame()
df_total = pd.DataFrame()

chrome_path = "./chromedriver.exe" #chromedriver.exe執行檔所存在的路徑
driver = webdriver.Chrome(chrome_path)
driver.get("https://www.ruten.com.tw/")

name_reach = "iphone充電線"
num_page = 100 #目標頁少一頁
num_product = 80

###  define 研究的產品 (罐頭)

    ### 搜尋名稱
driver.find_element_by_xpath("//input[@id='keyword']").send_keys(name_reach)
    ### 搜尋
driver.find_element_by_xpath("//input[@class='rt-header-search-submit']").click()
    
    ### 切換顯示方式
driver.find_element_by_xpath("//a[contains(@class,'list')]").click()
    
current_pag = 1
    
for pag in range(num_page-1) :
    print(current_pag) 
    url = driver.current_url
    
    ########################## 抓
    time.sleep(random.uniform(3,4))
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    
    
    
    Trace_num = []
    sale_num = []
    RY = []
    ID = []
    AD = []

    
    
    for pro in range(len(soup.find_all(class_ = "col3f"))):

        #sale_num
        try : 
            sale_num.append(int(soup.find_all(class_ = "col3f")[pro].find(class_ = "note").text.replace('銷售','')))
        except :
            sale_num.append('0')
        
        
        
        
        
        #紅黃框       
        RY.append((soup.find_all('dl','col3f')[pro].get('class')))
        
        #商品ID
        ID.append((soup.find_all('dl','col3f')[pro].get('_id')))
    
        #紅黃框   
        #for a in soup.find_all('dl','col3f'):
            #RY.append(a.get('class'))
            
        #商品ID
        #for a in soup.find_all('dl','col3f'):
            #ID.append(a.get('_id'))
    
    #追蹤人數
    if len(soup.find_all(title = re.compile('已有'))) > 80 :
        Trace_num = list(map(lambda x : x.text ,soup.find_all(title = re.compile('已有'))))[:80]
    else :
        Trace_num = list(map(lambda x : x.text ,soup.find_all(title = re.compile('已有'))))
    #廣告
    if len(soup.find_all(title = re.compile('已有'))) > 80 :
        AD = list(map(lambda x : x.text ,soup.find_all('span', class_='ads_tag')))[:num_product]
    else :
        AD = list(map(lambda x : x.text ,soup.find_all('span', class_='ads_tag')))
    ##########################
    df['ID'] = ID
    df['sale_num'] = sale_num
    df['AD'] = AD
    df['RY'] = RY
    df['Trace_num'] = Trace_num
    df['page'] = pag +1 
    
    
    df_total = pd.concat([df_total,df])
    
    current_pag += 1
    
    driver.find_element_by_xpath("//ul[@class='pages']//li[@class='next']//a").click()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [26]:
product_url = []

In [27]:
product_url = 'https://goods.ruten.com.tw/item/show?' + df_total['ID']

In [33]:
pd.DataFrame(product_url).to_csv('product_url.csv')

In [2]:
chrome_path = "./chromedriver.exe" #chromedriver.exe執行檔所存在的路徑
driver = webdriver.Chrome(chrome_path)

In [3]:
product_url = pd.read_csv('./product_url.csv')
product_url = product_url['ID']

In [4]:
result = [[] for _ in range(20)]
total = print(len(product_url))
current_pag = 1
for product in product_url[:1800] :
    print(str(current_pag) + '/' + str(len(product_url)))
    try :
        driver.get(product)
        elem = driver.find_element_by_xpath("//*")
        soup = bs.BeautifulSoup(elem.get_attribute("outerHTML"), 'html.parser')
        ######################################
        #產品名
        result[0].append(soup.find('span', class_='vmiddle').text)
        #圖片數
        try :
            imag = soup.find('ul', class_='item-gallery-thumbnail-list js-img-list')
            result[1].append(len (imag))
        except :
            result[1].append('None')
        #商品編號
        result[2].append(soup.find('span', class_='content').text)
        #直購價
        result[3].append(soup.find('strong', class_ = "rt-text-xx-large").text)
        #庫存
        result[4].append(soup.find('strong', class_="rt-text-isolated").text)
        #已賣數量 
        sell_num = soup.find('strong', class_='rt-text-x-large number')
        if sell_num != None:
            result[5].append(sell_num.text)
        else:
            result[5].append('0')
        #賣家!!
        se = soup.find('div', class_='seller-disc')
        seller_ID =  [s for s in se.stripped_strings][0]
        result[6].append(seller_ID)
        #評價!!
        try :
            reputation =  [s for s in se.stripped_strings][1]
            result[7].append(reputation)
        except :
            result[7].append('None')
        #問與答
        q = soup.find_all('span', class_ ='rt-text-parentesis count')
        QA = [qa for qa in q][0]
        result[8].append(QA.text)
        #購買人次
        buyer_num = [qa for qa in q][1]
        result[9].append(buyer_num.text)

         ###### #payment
        result[10].append(list(map(lambda x: x.text, soup.find_all('span', class_='vmiddle'))))
        #status
        result[11].append(soup.find('li', class_='status').text)
        #location
        result[12].append(soup.find('li', class_='location').text)
        #upload_time
        result[13].append(soup.find('li', class_='upload-time').text)
        #買家限制
        limitation = soup.find('li', class_="limitation")
        if limitation != None:
            result[14].append(limitation.find('span').text)
        else:
            result[14].append("0")
        #result[11].append(limitbuy(soup))
        #result[11].append((soup.find('li', class_="limitation")).text if limit != None else 'NA')
        #result[11].append(remain.text if remain != None else 'NA')
        #start_price
        result[15].append(soup.find('li', class_="initiation").find('span').text)
        #end
        result[16].append(soup.find('li', class_="putforward").text)
    ###### #delivery
        tmp = []
        for ele in map(lambda x: x.text.split('\n'), soup.find_all('ul', class_='detail-list')):
            tmp += ele
        result[17].append(list(filter(lambda x: len(x) > 0, map(lambda x: x.lstrip().rstrip(), tmp))))
        #all_sell_num
        try :
            result[18].append(soup.find('p', class_="seller-disc seller-disc-divider").find('strong').text)
        except :
            result[18].append('None')
        #update_time
        update_time = soup.find('span', class_="date")
        if update_time != None:
            result[19].append(update_time.text)
        else:
            result[19].append("0")    

        current_pag += 1
        
        
    except :
        pass
        current_pag += 1
    

7920
1/7920
2/7920
3/7920
4/7920
5/7920
6/7920
7/7920
8/7920
9/7920
10/7920
11/7920
12/7920
13/7920
14/7920
15/7920
16/7920
17/7920
18/7920
19/7920
20/7920
21/7920
22/7920
23/7920
24/7920
25/7920
26/7920
27/7920
28/7920
29/7920
30/7920
31/7920
32/7920
33/7920
34/7920
35/7920
36/7920
37/7920
38/7920
39/7920
40/7920
41/7920
42/7920
43/7920
44/7920
45/7920
46/7920
47/7920
48/7920
49/7920
50/7920
51/7920
52/7920
53/7920
54/7920
55/7920
56/7920
57/7920
58/7920
59/7920
60/7920
61/7920
62/7920
63/7920
64/7920
65/7920
66/7920
67/7920
68/7920
69/7920
70/7920
71/7920
72/7920
73/7920
74/7920
75/7920
76/7920
77/7920
78/7920
79/7920
80/7920
81/7920
82/7920
83/7920
84/7920
85/7920
86/7920
87/7920
88/7920
89/7920
90/7920
91/7920
92/7920
93/7920
94/7920
95/7920
96/7920
97/7920
98/7920
99/7920
100/7920
101/7920
102/7920
103/7920
104/7920
105/7920
106/7920
107/7920
108/7920
109/7920
110/7920
111/7920
112/7920
113/7920
114/7920
115/7920
116/7920
117/7920
118/7920
119/7920
120/7920
121/7920
122/7920
123/7

923/7920
924/7920
925/7920
926/7920
927/7920
928/7920
929/7920
930/7920
931/7920
932/7920
933/7920
934/7920
935/7920
936/7920
937/7920
938/7920
939/7920
940/7920
941/7920
942/7920
943/7920
944/7920
945/7920
946/7920
947/7920
948/7920
949/7920
950/7920
951/7920
952/7920
953/7920
954/7920
955/7920
956/7920
957/7920
958/7920
959/7920
960/7920
961/7920
962/7920
963/7920
964/7920
965/7920
966/7920
967/7920
968/7920
969/7920
970/7920
971/7920
972/7920
973/7920
974/7920
975/7920
976/7920
977/7920
978/7920
979/7920
980/7920
981/7920
982/7920
983/7920
984/7920
985/7920
986/7920
987/7920
988/7920
989/7920
990/7920
991/7920
992/7920
993/7920
994/7920
995/7920
996/7920
997/7920
998/7920
999/7920
1000/7920
1001/7920
1002/7920
1003/7920
1004/7920
1005/7920
1006/7920
1007/7920
1008/7920
1009/7920
1010/7920
1011/7920
1012/7920
1013/7920
1014/7920
1015/7920
1016/7920
1017/7920
1018/7920
1019/7920
1020/7920
1021/7920
1022/7920
1023/7920
1024/7920
1025/7920
1026/7920
1027/7920
1028/7920
1029/7920
1030/79

1750/7920
1751/7920
1752/7920
1753/7920
1754/7920
1755/7920
1756/7920
1757/7920
1758/7920
1759/7920
1760/7920
1761/7920
1762/7920
1763/7920
1764/7920
1765/7920
1766/7920
1767/7920
1768/7920
1769/7920
1770/7920
1771/7920
1772/7920
1773/7920
1774/7920
1775/7920
1776/7920
1777/7920
1778/7920
1779/7920
1780/7920
1781/7920
1782/7920
1783/7920
1784/7920
1785/7920
1786/7920
1787/7920
1788/7920
1789/7920
1790/7920
1791/7920
1792/7920
1793/7920
1794/7920
1795/7920
1796/7920
1797/7920
1798/7920
1799/7920
1800/7920


In [5]:
df = pd.DataFrame(result).T
df.columns = ['name','image_num','productID','price','inventory','sell_num','seller_ID','reputation','QA', 'buyer_num','payment','status','location','upload-time','buylimit','start_price','end','delivery','all_sell_num','update-time']

In [6]:
df.to_csv('0-1800.csv',en)

,name,image_num,productID,price,inventory,sell_num,seller_ID,reputation,QA,buyer_num,payment,status,location,upload-time,buylimit,start_price,end,delivery,all_sell_num,update-time
0,保證 最高規 iPhone 11 X 8 7 6 充電線 傳輸線 for iPhone iP...,9,21306013652122,$180,59,37126,qqn78,49922,1867,18888,[保證 最高規 iPhone 11 X 8 7 6 充電線 傳輸線 for iPhone i...,物品狀況： 全新,物品所在地： 台灣.新北市,上架時間： 2013-06-01 00:54:08,0,$180元,可能會提前結束販售,"[支援付款快手, Pi 拍錢包X支付連, 最高P幣 4.5% 回饋(說明)PChomePay...",45,2019-10-22 00:24:07
1,MCDODO QC4.0 iPhone TypeC 快充線 捲線 彈簧線 180cm 快速充...,9,21919391256750,$199,99,4302,xnetcr95277,31249,2,103,[MCDODO QC4.0 iPhone TypeC 快充線 捲線 彈簧線 180cm 快速...,物品狀況： 全新,物品所在地： 台灣.桃園市,上架時間： 2019-05-08 15:52:28,0,$199元,可能會提前結束販售,"[支援付款快手, Pi 拍錢包X支付連, 最高P幣 4.5% 回饋(說明)PChomePay...",154,2019-07-24 20:53:07
2,【防止過充！自動斷電充電線】蘋果專用 編織不打結 iphone 充電線 快充2.4A lig...,9,21904684329839,$269 - $288,99,1604,dodo790119,126778,14,482,[【防止過充！自動斷電充電線】蘋果專用 編織不打結 iphone 充電線 快充2.4A li...,物品狀況： 全新,物品所在地： 台灣.新北市,上架時間： 2019-01-24 11:37:30,0,$269 - 288元,可能會提前結束販售,"[支援付款快手, Pi 拍錢包X支付連, 最高P幣 4.5% 回饋(說明)PChomePay...",927,2019-11-07 10:17:31
3,蘋果 Apple快充【PD+QC3.0快充組】mcdodo 麥多多 30W快充頭+快充線 i...,9,21707288442128,$550 - $790,54,111,mmxcdboy123,3362,16,110,[蘋果 Apple快充【PD+QC3.0快充組】mcdodo 麥多多 30W快充頭+快充線 ...,物品狀況： 全新,物品所在地： 台灣.台北市,上架時間： 2017-02-19 09:45:14,0,$550 - 790元,可能會提前結束販售,"[賣家保固, 專櫃正品, 可開收據, 附保證書, 有多種尺寸, 支援付款快手, Pi 拍錢包...",294,2019-12-24 15:44:12
4,【最新】360度無死角超吸快充 磁吸充電線 蘋果/安卓/Type-c 充電線 iphone充...,9,21807687309214,$69 - $199,99,14571,jason810046,14592,651,6117,[【最新】360度無死角超吸快充 磁吸充電線 蘋果/安卓/Type-c 充電線 iphone...,物品狀況： 全新,物品所在地： 台灣.桃園市,上架時間： 2018-02-16 15:20:16,近半年棄標次數 ≦ 3次,$69 - 199元,可能會提前結束販售,"[賣家保固, 公司貨, 有多種尺寸, Pi 拍錢包X支付連, 最高P幣 4.5% 回饋(說明...",71,2019-11-12 10:01:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,蘋果 iPhone 傳輸線 充電線 線 Apple線 iphone充電線 數據線 iPhon...,3,21728717712780,$125,15,5,st9761,1191,0,8,[蘋果 iPhone 傳輸線 充電線 線 Apple線 iphone充電線 數據線 iPho...,物品狀況： 全新,物品所在地： 台灣.台北市,上架時間： 2017-07-16 17:24:05,近半年棄標次數 ≦ 2次,$125元,可能會提前結束販售,[PChomePay支付連 現金 (ATM、餘額、銀行支付)PChome國際連儲值餘額儲值7...,209,2018-01-25 22:34:51
1792,iphone原廠豆腐頭 充電器 原廠傳輸線 iphone 充電 傳輸線 豆腐頭 iphone...,4,21728690279205,$260,99,3,appledac,45,0,2,[iphone原廠豆腐頭 充電器 原廠傳輸線 iphone 充電 傳輸線 豆腐頭 iphon...,物品狀況： 全新,物品所在地： 台灣.台北市,上架時間： 2017-07-12 14:41:23,0,$260元,可能會提前結束販售,"[原廠保固, 有多種尺寸, 有多種顏色, Pi 拍錢包X支付連, 最高P幣 2.5% 回饋(...",1272,2017-07-16 12:35:00
1793,Mophie Powerstation Plus 6000mAh 行動電源 內建iPhone...,3,21728688069827,"$3,126",21,0,linmashcee,4303,0,0,[Mophie Powerstation Plus 6000mAh 行動電源 內建iPhon...,物品狀況： 全新,物品所在地： 台灣.台北市,上架時間： 2017-07-12 00:17:07,0,"$3,290元",可能會提前結束販售,"[支援付款快手, Pi 拍錢包X支付連, 最高P幣 4.5% 回饋(說明)PChomePay...",1206,2018-11-21 16:41:51
1794,(JL) 蘋果數據線iphone6充電線器2米單頭7手機6s加長5s短5認證Plus快充,1,21724486961505,$200,99,0,mark19520221,2297,0,0,"[(JL) 蘋果數據線iphone6充電線器2米單頭7手機6s加長5s短5認證Plus快充,...",物品狀況： 全新,物品所在地： 台灣.台北市,上架時間： 2017-06-13 16:02:51,0,$200元,可能會提前結束販售,"[支援付款快手, PChomePay支付連 現金 (ATM、餘額、銀行支付)7-11取貨付款...",11623,0


In [ ]:
df_total['rank_eachpage'] = df_total.index+1
df_total.to_csv('./0117.csv',index=False)

In [31]:
result = [[] for _ in range(20)]
total = print(len(product_url))
current_pag = 1
for product in product_url :
    print(str(current_pag) + '/' + str(len(product_url)))
    driver.get(product)
    elem = driver.find_element_by_xpath("//*")
    soup = bs.BeautifulSoup(elem.get_attribute("outerHTML"), 'html.parser')
    ######################################
    #產品名
    result[0].append(soup.find('span', class_='vmiddle').text)
    #圖片數
    try :
        imag = soup.find('ul', class_='item-gallery-thumbnail-list js-img-list')
        result[1].append(len (imag))
    except :
        result[1].append('None')
    #商品編號
    result[2].append(soup.find('span', class_='content').text)
    #直購價
    result[3].append(soup.find('strong', class_ = "rt-text-xx-large").text)
    #庫存
    result[4].append(soup.find('strong', class_="rt-text-isolated").text)
    #已賣數量 
    sell_num = soup.find('strong', class_='rt-text-x-large number')
    if sell_num != None:
        result[5].append(sell_num.text)
    else:
        result[5].append('0')
    #賣家!!
    se = soup.find('div', class_='seller-disc')
    seller_ID =  [s for s in se.stripped_strings][0]
    result[6].append(seller_ID)
    #評價!!
    reputation =  [s for s in se.stripped_strings][1]
    result[7].append(reputation)
    #問與答
    q = soup.find_all('span', class_ ='rt-text-parentesis count')
    QA = [qa for qa in q][0]
    result[8].append(QA.text)
    #購買人次
    buyer_num = [qa for qa in q][1]
    result[9].append(buyer_num.text)
    
     ###### #payment
    result[10].append(list(map(lambda x: x.text, soup.find_all('span', class_='vmiddle'))))
    #status
    result[11].append(soup.find('li', class_='status').text)
    #location
    result[12].append(soup.find('li', class_='location').text)
    #upload_time
    result[13].append(soup.find('li', class_='upload-time').text)
    #買家限制
    limitation = soup.find('li', class_="limitation")
    if limitation != None:
        result[14].append(limitation.find('span').text)
    else:
        result[14].append("0")
    #result[11].append(limitbuy(soup))
    #result[11].append((soup.find('li', class_="limitation")).text if limit != None else 'NA')
    #result[11].append(remain.text if remain != None else 'NA')
    #start_price
    result[15].append(soup.find('li', class_="initiation").find('span').text)
    #end
    result[16].append(soup.find('li', class_="putforward").text)
###### #delivery
    tmp = []
    for ele in map(lambda x: x.text.split('\n'), soup.find_all('ul', class_='detail-list')):
        tmp += ele
    result[17].append(list(filter(lambda x: len(x) > 0, map(lambda x: x.lstrip().rstrip(), tmp))))
    #all_sell_num
    result[18].append(soup.find('p', class_="seller-disc seller-disc-divider").find('strong').text)
    #update_time
    update_time = soup.find('span', class_="date")
    if update_time != None:
        result[19].append(update_time.text)
    else:
        result[19].append("0")    
        
    current_pag += 1
    

7920
1/7920
2/7920
3/7920
4/7920
5/7920
6/7920
7/7920
8/7920
9/7920
10/7920
11/7920
12/7920
13/7920
14/7920
15/7920
16/7920
17/7920
18/7920
19/7920
20/7920
21/7920
22/7920
23/7920
24/7920
25/7920
26/7920
27/7920
28/7920
29/7920
30/7920
31/7920
32/7920
33/7920
34/7920
35/7920
36/7920
37/7920
38/7920
39/7920
40/7920
41/7920
42/7920
43/7920
44/7920
45/7920
46/7920
47/7920
48/7920
49/7920
50/7920
51/7920
52/7920
53/7920
54/7920
55/7920
56/7920
57/7920
58/7920
59/7920
60/7920
61/7920
62/7920
63/7920
64/7920
65/7920
66/7920
67/7920
68/7920
69/7920
70/7920
71/7920
72/7920
73/7920
74/7920
75/7920
76/7920
77/7920
78/7920
79/7920
80/7920
81/7920
82/7920
83/7920
84/7920
85/7920
86/7920
87/7920
88/7920
89/7920
90/7920
91/7920
92/7920
93/7920
94/7920
95/7920
96/7920
97/7920
98/7920
99/7920
100/7920
101/7920
102/7920
103/7920
104/7920
105/7920
106/7920
107/7920
108/7920
109/7920
110/7920
111/7920
112/7920
113/7920
114/7920
115/7920
116/7920
117/7920
118/7920
119/7920
120/7920
121/7920
122/7920
123/7

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(result).T
df.columns = ['name','image_num','productID','price','inventory','sell_num','seller_ID','reputation','QA', 'buyer_num','payment','status','location','upload-time','buylimit','start_price','end','delivery','all_sell_num','update-time']

In [8]:
df_total['rank_eachpage'] = df_total.index+1
df_total.to_csv('./0117.csv',index=False)

In [27]:
len(Trace_num)

80

In [28]:
len(sale_num) 

80

In [29]:
len(RY)

80

In [30]:
len(ID)

80

In [31]:
len(AD)

80

In [32]:
df_total['rank_eachpage'] = df_total.index+1
df_total.to_csv('./0117.csv',index=False)

In [34]:
pd.read_csv('./0117.csv')

,ID,sale_num,AD,RY,Trace_num,page,rank_eachpage
0,21306013652122,37123,廣告,"['col3f', 's_shining']",591,1,1
1,21919391256750,4302,廣告,['col3f'],41,1,2
2,21904684329839,1602,廣告,['col3f'],224,1,3
3,21707288442128,111,廣告,['col3f'],56,1,4
4,21807687309214,14568,廣告,"['col3f', 's_shining']",4800,1,5
...,...,...,...,...,...,...,...
2395,21848073055890,24,NaN,['col3f'],8,30,76
2396,21917115214903,33,NaN,['col3f'],7,30,77
2397,21827294445693,18,NaN,['col3f'],5,30,78
2398,21910116907388,261,NaN,['col3f'],39,30,79
